In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

warnings.filterwarnings(action='ignore')

if get_ipython():
    print("jupyter envirionment")
    PROJECT_DIR = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
    from tqdm import tqdm_notebook as tqdm  # 이거 안해주면 한 줄씩 출력됨 ;;
else:
    print("command shell envirionment")
    PROJECT_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    from tqdm import tqdm

# sys.pathdp PROJECT_DIR 추가
sys.path.insert(0, PROJECT_DIR)

jupyter envirionment


In [3]:
DATA_DIR = PROJECT_DIR + '/ext/data'
RAW_DATA_DIR = DATA_DIR + '/raw'

In [38]:
with open('./data_news/train.jsonl', 'r',encoding='utf-8') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)
    
train_df = pd.DataFrame(trains)
train_df.head()

,media,id,article_original,abstractive,extractive
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


In [42]:
#dataframe에 extractive 숫자 답안을 본래 문장으로 추가
ext_ans = []
for i in range(len(train_df)):
    temp =[]
    for num in train_df.extractive[i]:
        temp.append(train_df.article_original[i][num])
    ext_ans.append(temp)
train_df['ext_ans'] = ext_ans

In [45]:
train_df.head()

,media,id,article_original,abstractive,extractive,ext_ans
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]",[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]",[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]",[16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가...
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]",[대구·경북첨단의료산업진흥재단(이하 대구첨복재단) 의약생산센터는 주세제 특수제제인 ...
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]",[식품의약품안전처는 29일 여름철 어린이가 즐겨 마시는 탄산음료 282개와 혼합음료...


In [117]:
# 기사를 리스트가 아닌 한 문단의 스트링으로 만드는 함수
def article_strfy(corpus_list,article_no):
    article = ''
    for sent in corpus_list[article_no]:
        article += str(sent)
        article += ' '
    return article, article_no   #합쳐진 기사와 answer을 리턴한다.

In [120]:
print(article_strfy(train_df['article_original'],1))

('미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다. LA 에인절스의 최지만은 맹활약으로 주전 입지를 굳혀가는 반면 텍사스 레인저스의 추신수는 개막전 선발에서 밀려나는 수모를 당한 뒤에서도 좀처럼 반등을 하지 못하고 있다. 최지만은 2일(한국시간) 미국 플로리다주 세인트피터즈버그 트로피카나 필드에서 열린 2019 미국프로야구 메이저리그 콜로라도 로키스와 홈경기에 3번 타자 1루수로 선발 출전해 4타수 2안타를 치고 1득점을 올렸다. 시즌 타율은 0.250에서 0.313(16타수 5안타)로 올랐다. 최지만은 개막전에서 4타수 무안타로 침묵했으나 이후 3경기 연속 안타이자 2경기 연속 멀티 히트를 뽑아내며 입지를 굳혀갔다. 2016년 LA 에인절스 소속으로 메이저리그에 데뷔한 최지만이 2경기 연속 멀티 히트를 친 것은 이번이 처음이다. 스코어가 벌어진 탓에 콜로라도 불펜의 필승조인 오승환과 최지만의 맞대결은 성사되지 않았다. 추신수는 들쭉날쭉한 출전 일정 탓인지 타격감을 찾지 못하고 있다. 추신수는 이날 텍사스주 알링턴 글로브 라이프 파크에서 열린 휴스턴 애스트로스와 홈경기에 1번 지명타자로 선발 출전했지만 4타수 무안타로 침묵했다. 시즌 타율은 0.167에서 0.100(10타수 1안타)로 떨어졌다. 추신수는 11년 만에 개막전 선발 라인업에서 제외되는 등 이날까지 텍사스가 치른 4경기 가운데 2경기에만 선발 출전했다. 크리스 우드워드 텍사스 감독은 상대 선발이 좌완이면 좌타자인 추신수 대신 우타자인 헌터 펜스를 선발 지명타자로 기용하고 있다. 텍사스(2승 2패)는 휴스턴 마운드에 2안타로 꽁꽁 묶이며 1-2로 패해 2연승 행진을 마감했다. 한편 피츠버그 파이리츠의 강정호는 914일 만에 홈경기에 나섰지만 안타를 만들어내지 못했다. 이날 펜실베이니아주 피츠버그의 PNC 파크에서 열린 세인트루이스 카디널스와 홈 개막전에 선발 라인업에서 제외된 강정호는 벤치에서 경기를 시작했다. 강정호는 4-4 동점이 된 8회 초 2사 2루에서 3

### Gensim

In [127]:
#gensim summarization
from gensim.summarization import summarize
from gensim.summarization import summarize_corpus
article, _ = article_strfy(train_df['article_original'],1)
summarize(article,.2)

'LA 에인절스의 최지만은 맹활약으로 주전 입지를 굳혀가는 반면 텍사스 레인저스의 추신수는 개막전 선발에서 밀려나는 수모를 당한 뒤에서도 좀처럼 반등을 하지 못하고 있다.\n최지만은 2일(한국시간) 미국 플로리다주 세인트피터즈버그 트로피카나 필드에서 열린 2019 미국프로야구 메이저리그 콜로라도 로키스와 홈경기에 3번 타자 1루수로 선발 출전해 4타수 2안타를 치고 1득점을 올렸다.\n추신수는 이날 텍사스주 알링턴 글로브 라이프 파크에서 열린 휴스턴 애스트로스와 홈경기에 1번 지명타자로 선발 출전했지만 4타수 무안타로 침묵했다.\n연장 10회 말 1사에서 다시 타석에 들어선 강정호는 세인트루이스의 우완 ‘파이어볼러’ 조던 힉스에게 3구 삼진으로 물러났다.'

### Lexrank

In [121]:
#lexrank
from lexrankr import LexRank
lexrank = LexRank()

article, no = article_strfy(train_df.article_original,1)
              
lexrank.summarize(article)
summaries = lexrank.probe(3)  #3을 넣으면 3문장 요약

print(article)
print(summaries)
print(train_df['ext_ans'][no])

미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다. LA 에인절스의 최지만은 맹활약으로 주전 입지를 굳혀가는 반면 텍사스 레인저스의 추신수는 개막전 선발에서 밀려나는 수모를 당한 뒤에서도 좀처럼 반등을 하지 못하고 있다. 최지만은 2일(한국시간) 미국 플로리다주 세인트피터즈버그 트로피카나 필드에서 열린 2019 미국프로야구 메이저리그 콜로라도 로키스와 홈경기에 3번 타자 1루수로 선발 출전해 4타수 2안타를 치고 1득점을 올렸다. 시즌 타율은 0.250에서 0.313(16타수 5안타)로 올랐다. 최지만은 개막전에서 4타수 무안타로 침묵했으나 이후 3경기 연속 안타이자 2경기 연속 멀티 히트를 뽑아내며 입지를 굳혀갔다. 2016년 LA 에인절스 소속으로 메이저리그에 데뷔한 최지만이 2경기 연속 멀티 히트를 친 것은 이번이 처음이다. 스코어가 벌어진 탓에 콜로라도 불펜의 필승조인 오승환과 최지만의 맞대결은 성사되지 않았다. 추신수는 들쭉날쭉한 출전 일정 탓인지 타격감을 찾지 못하고 있다. 추신수는 이날 텍사스주 알링턴 글로브 라이프 파크에서 열린 휴스턴 애스트로스와 홈경기에 1번 지명타자로 선발 출전했지만 4타수 무안타로 침묵했다. 시즌 타율은 0.167에서 0.100(10타수 1안타)로 떨어졌다. 추신수는 11년 만에 개막전 선발 라인업에서 제외되는 등 이날까지 텍사스가 치른 4경기 가운데 2경기에만 선발 출전했다. 크리스 우드워드 텍사스 감독은 상대 선발이 좌완이면 좌타자인 추신수 대신 우타자인 헌터 펜스를 선발 지명타자로 기용하고 있다. 텍사스(2승 2패)는 휴스턴 마운드에 2안타로 꽁꽁 묶이며 1-2로 패해 2연승 행진을 마감했다. 한편 피츠버그 파이리츠의 강정호는 914일 만에 홈경기에 나섰지만 안타를 만들어내지 못했다. 이날 펜실베이니아주 피츠버그의 PNC 파크에서 열린 세인트루이스 카디널스와 홈 개막전에 선발 라인업에서 제외된 강정호는 벤치에서 경기를 시작했다. 강정호는 4-4 동점이 된 8회 초 2사 2루에서 3루 

### TfidfVectorizer로 정제 후 학습

In [130]:
#from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test = train_test_split(train_df['article_original'],train_df['ext_ans'])


In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(lowercase='False',ngram_range=(1,2))
tfidf_vect.fit(train_df['article_original'])
X_tfidf = tfidf_vect.transform(train_df['article_original'])
Y_tfidf = tfidf_vect.transform(train_df['ext_ans'])
X_tfidf.shape, Y_tfidf.shape

AttributeError: 'list' object has no attribute 'lower'